In [1]:
from ACDC.random_walk_classifier import * 
from ACDC.cell_type_annotation import * 

In [2]:
import time
import pandas as pd
import numpy as np
from collections import Counter
import pickle

channels = ['CD45','CD45RA', 'CD19', 'CD11b', 'CD4', 'CD8', 'CD34',
           'CD20', 'CD33', 'CD123', 'CD38', 'CD90', 'CD3']

path = 'data/BMMC_benchmark/'

df = pd.read_csv(path + 'BMMC_benchmark.csv.gz', sep=',', header = 0, compression = 'gzip')
df = df[df.cell_type != 'NotGated']


table = pd.read_csv(path + 'BMMC_table.csv', sep=',', header=0, index_col=0)
table = table.fillna(0)

cts, channels = get_label(table)

X0= np.arcsinh((df[channels].values - 1.0)/5.0)

In [3]:
idx2ct = [key for idx, key in enumerate(table.index)]
idx2ct.append('unknown')

ct2idx = {key:idx for idx, key in enumerate(table.index)}
ct2idx['unknown'] = len(table.index)
        
ct_score = np.abs(table.as_matrix()).sum(axis = 1)

## compute manual gated label
y0 = np.zeros(df.cell_type.shape)

for i, ct in enumerate(df.cell_type):
    if ct in ct2idx:
        y0[i] = ct2idx[ct]
    else:
        y0[i] = ct2idx['unknown']

In [4]:
from sklearn.metrics import accuracy_score, confusion_matrix
import phenograph


thres = 0.5


In [5]:
tic = time.clock()
phenograph.cluster(X0)
toc = time.clock()
print(toc-tic)

Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 16.506507873535156 seconds
Jaccard graph constructed in 6.77113676071167 seconds
Wrote graph to binary file in 4.350315093994141 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.87792
After 10 runs, maximum modularity is Q = 0.879066
Louvain completed 30 runs in 174.9412968158722 seconds
PhenoGraph complete in 203.71706223487854 seconds
125.84516500000001


In [6]:


result = []
score_final = []
process_time = []

    
X = X0
y_true = y0

tic = time.clock()
mk_model =  compute_marker_model(pd.DataFrame(X, columns = channels), table, 0.0)

## compute posterior probs
score = get_score_mat(X, [], table, [], mk_model)
score = np.concatenate([score, 1.0 - score.max(axis = 1)[:, np.newaxis]], axis = 1)    

## get indices     
ct_index = get_unique_index(X, score, table, thres)
    

## running ACDC
tic = time.clock()
res_c = get_landmarks(X, score, ct_index, idx2ct, phenograph, thres)

landmark_mat, landmark_label = output_feature_matrix(res_c, [idx2ct[i] for i in range(len(idx2ct))]) 

landmark_label = np.array(landmark_label)

toc = time.clock()

time0 = toc - tic

Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 1.8151025772094727 seconds
Jaccard graph constructed in 1.1808147430419922 seconds
Wrote graph to binary file in 0.5162525177001953 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.649863
After 2 runs, maximum modularity is Q = 0.651331
After 4 runs, maximum modularity is Q = 0.65239
After 9 runs, maximum modularity is Q = 0.654965
Louvain completed 29 runs in 10.2366361618042 seconds
PhenoGraph complete in 13.908045291900635 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 0.5077486038208008 seconds
Jaccard graph constructed in 0.649843692779541 seconds
Wrote graph to binary file in 0.32376813888549805 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.600011
After 2 runs, maximum modularity is Q = 0.603739
After 5 runs, maximum modularity is Q = 0.607146
Afte

In [7]:
score_final = []
process_time = []
for n_neighbor in [10, 20, 30]:
    tic = time.clock()
    
    lp, y_pred = rm_classify(X, landmark_mat, landmark_label, n_neighbor)
    
    toc = time.clock()
    time1 = toc - tic
    
    process_time.append(time0 + time1)

    score_final.append(accuracy_score(y_true, [ct2idx[c] for c in y_pred]))    
    
print(process_time)
print(score_final)

[245.519975, 309.009604, 376.5867919999999]
[0.92029065286799516, 0.92245586994018125, 0.92498807295680574]


In [8]:
X.shape

(81747, 13)